In [1]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 33.0 MB/s eta 0:00:00


In [2]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


In [3]:
def extract_data(path):
  return pd.read_csv(path, sep = '\t', quoting=csv.QUOTE_NONE) 

In [4]:
def data_bal(path_train, path_test):
  df = extract_data(path_train)
  dft = extract_data(path_test)

  df_full = df.append(dft)

  data_1 = df_full[df_full['Quality']==1]
  data_2 = df_full[df_full['Quality']==0]

  df_bal = data_2.append(data_1[:1900])

  df_bal = df_bal.sample(frac = 1)
  df_bal.reset_index(inplace=True)

  return df_bal


In [5]:
def model_pipe():
  
  bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

  input_text1 = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text1')
  input_text2 = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input_text2')
  preprocessed_text1 = bert_preprocess(input_text1)
  preprocessed_text2 = bert_preprocess(input_text2)
  out1 = bert_encoder(preprocessed_text1)
  out2 = bert_encoder(preprocessed_text2)
  concatenated = tf.keras.layers.Concatenate(axis=-1)([out1['pooled_output'], out2['pooled_output']])
  dense1 = tf.keras.layers.Dense(256, activation='relu')(concatenated)
  dense2 = tf.keras.layers.Dense(128, activation='relu')(dense1)
  dropout1 = tf.keras.layers.Dropout(0.2)(dense2)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout1)

  model = tf.keras.Model(inputs=[input_text1,input_text2], outputs=output)

  METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=METRICS)

  return model

In [6]:
def model_train(path_train, path_test):

  df_bal = data_bal(path_train, path_test)

  X_train, X_test, y_train, y_test = train_test_split(df_bal[['#1 String','#2 String']],df_bal['Quality'], stratify= df_bal['Quality'],test_size = 0.15)

  model = model_pipe()
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  history = model.fit([X_train['#1 String'], X_train['#2 String']], y_train, epochs=7, batch_size=32,callbacks=[callback], validation_split=0.15)

  return history,model,X_test,y_test

  

In [7]:
history,model,X_test,y_test = model_train('/content/drive/MyDrive/msrp_project/MSRParaphraseCorpus/msr_paraphrase_train.txt',
                      '/content/drive/MyDrive/msrp_project/MSRParaphraseCorpus/msr_paraphrase_test.txt')

<ipython-input-4-74e2d7981acc>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full = df.append(dft)
<ipython-input-4-74e2d7981acc>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bal = data_2.append(data_1[:1900])


Epoch 1/7
86/86 [==============================] - 83s 772ms/step - loss: 0.7237 - accuracy: 0.5082 - precision: 0.5076 - recall: 0.5113 - val_loss: 0.6959 - val_accuracy: 0.5031 - val_precision: 0.5031 - val_recall: 0.9877
Epoch 2/7
86/86 [==============================] - 66s 772ms/step - loss: 0.6920 - accuracy: 0.5373 - precision: 0.5384 - recall: 0.5164 - val_loss: 0.6818 - val_accuracy: 0.5649 - val_precision: 0.5587 - val_recall: 0.6434
Epoch 3/7
86/86 [==============================] - 66s 769ms/step - loss: 0.6902 - accuracy: 0.5370 - precision: 0.5355 - recall: 0.5507 - val_loss: 0.6866 - val_accuracy: 0.5711 - val_precision: 0.6385 - val_recall: 0.3402
Epoch 4/7
86/86 [==============================] - 66s 774ms/step - loss: 0.6886 - accuracy: 0.5381 - precision: 0.5378 - recall: 0.5346 - val_loss: 0.6795 - val_accuracy: 0.5814 - val_precision: 0.6289 - val_recall: 0.4098
Epoch 5/7
86/86 [==============================] - 67s 780ms/step - loss: 0.6841 - accuracy: 0.5585 - pr

In [9]:
y_predicted = model.predict([X_test['#1 String'], X_test['#2 String']])
y_predicted = y_predicted.flatten()
import numpy as np
y_predicted = np.where(y_predicted > 0.5, 1, 0)

from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
print(cm)
print(classification_report(y_test, y_predicted)) 

18/18 [==============================] - 14s 646ms/step
[[170 116]
 [122 163]]
              precision    recall  f1-score   support

           0       0.58      0.59      0.59       286
           1       0.58      0.57      0.58       285

    accuracy                           0.58       571
   macro avg       0.58      0.58      0.58       571
weighted avg       0.58      0.58      0.58       571

